# 上週作業

## 這個部份上周剛好講過，所以跳過，自己看。

In [1]:
import os

os.chdir("../../")

In [ ]:
import json

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

from src.io.path_definition import get_project_dir


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Ingredients: {input}\nOrigin: {output}",
)

with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

examples = []

for recipe in recipe_train:
    examples.append({"input": " ".join(recipe['ingredients']),
                     "output": recipe['cuisine']})

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=5,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Find the recipe origin based on the ingredients",
    suffix="Ingredients: {ingredients}\nOrigin:",
    input_variables=["ingredients"],
)

In [ ]:
similar_prompt

In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

existing_ingredients = recipe_test[99]['ingredients']

similar_prompt.invoke(", ".join(existing_ingredients))

In [5]:
from langchain.chat_models import ChatOpenAI

from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

C:\Users\MengChieh\Miniconda3\envs\llm_examples\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
chain = similar_prompt|model

In [ ]:
chain.invoke(", ".join(existing_ingredients))

##  飛安報告

In [3]:
from src.io.path_definition import get_project_dir

import pandas as pd

df = pd.read_excel(os.path.join(get_project_dir(), 'tutorial', 'Week-3', 'Data sample.xlsx'))

In [ ]:
# system_prompt = PromptTemplate.from_template(
#     '''You are an AI assistant assigned with a task of safty report classification based on the content.
#     You are a seasoned flight safety inspector with deep and extensive knowledge of aviation safty. 
    
#     You always do the best work you can. You are highly analytical and pay close attention to details. 
    
#     The candidates of the output are:

#     - `Organizational Influence;Resource Management`
#     - `Organizational Influence;Organizational Climate`
#     - `Organizational Influence;rganizational Process`
#     - `Unsafe Supervisions;Inadequate Supervision`
#     - `Unsafe Supervisions;Planned Inappropriate Operations`
#     - `Unsafe Supervisions;Failed to Correct Problem`
#     - `Unsafe Supervisions;Supervisory Violation`
#     - `Precondition for Unsafe Acts;Environmental Factors;Physical Environment`
#     - `Precondition for Unsafe Acts;Environmental Factors;Technological Environment`
#     - `Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State`
#     - `Precondition for Unsafe Acts;Condition of Operators;Adverse Physiological State`
#     - `Precondition for Unsafe Acts;Condition of Operators;Physical/Mental Limitations`
#     - `Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management`
#     - `Precondition for Unsafe Acts;Personnel Factors;Personal Readiness`
#     - `Unsafe Acts;Errors;Decision Errors`
#     - `Unsafe Acts;Errors;Skill-Based Errors`
#     - `Unsafe Acts;Errors;Perceptual Errors`
#     - `Unsafe Acts;Violations;Routine`
#     - `Unsafe Acts;Violations;Exceptional`

#     The output is from one of the candidates. 
#     ''')

# system_message = SystemMessagePromptTemplate(prompt=system_prompt)

# response_schemas = [
#         ResponseSchema(name="category", description="The predicted category of the classification")
#     ]


# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# format_instructions = output_parser.get_format_instructions()

# human_prompt = PromptTemplate(template='{report}; format instruction: {format_instructions}',
#                               input_variables=["report"],
#                               partial_variables={'format_instructions': format_instructions}
#                               )
# human_message = HumanMessagePromptTemplate(prompt=human_prompt)

# chat_prompt = ChatPromptTemplate.from_messages([system_message,
#                                                 human_message
#                                                ])

# chain = chat_prompt|model|output_parser

### 回家作業 1

若要飛安事故報告可以有複數分類結果，如何調整Prompt，包含parser

In [6]:
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


system_prompt = PromptTemplate.from_template(
    '''You are an AI assistant assigned with a task of safty report classification based on the content.
    You are a seasoned flight safety inspector with deep and extensive knowledge of aviation safty. 
    
    You always do the best work you can. You are highly analytical and pay close attention to details. 
    
    The candidates of the output are:

    - `Organizational Influence;Resource Management`
    - `Organizational Influence;Organizational Climate`
    - `Organizational Influence;rganizational Process`
    - `Unsafe Supervisions;Inadequate Supervision`
    - `Unsafe Supervisions;Planned Inappropriate Operations`
    - `Unsafe Supervisions;Failed to Correct Problem`
    - `Unsafe Supervisions;Supervisory Violation`
    - `Precondition for Unsafe Acts;Environmental Factors;Physical Environment`
    - `Precondition for Unsafe Acts;Environmental Factors;Technological Environment`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Physiological State`
    - `Precondition for Unsafe Acts;Condition of Operators;Physical/Mental Limitations`
    - `Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management`
    - `Precondition for Unsafe Acts;Personnel Factors;Personal Readiness`
    - `Unsafe Acts;Errors;Decision Errors`
    - `Unsafe Acts;Errors;Skill-Based Errors`
    - `Unsafe Acts;Errors;Perceptual Errors`
    - `Unsafe Acts;Violations;Routine`
    - `Unsafe Acts;Violations;Exceptional`

     The report can involve multiple categories.
    ''')

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

response_schemas = [
        ResponseSchema(name="categories", description="The predicted categories as a python list")
    ]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

human_prompt = PromptTemplate(template='{report}; format instruction: {format_instructions}',
                              input_variables=["report"],
                              partial_variables={'format_instructions': format_instructions}
                              )
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                               ])

chain = chat_prompt|model|output_parser

In [7]:
text = df.iloc[3]['Report 1']
print(text)

Aircraft X was inbound to the airport from the southeast. Apparently the aircraft was on an IFR flight plan and I completely overlooked that information. It took the aircraft traveling about 10 miles inside my airspace before the pilot called and made first contact with me. When the pilot finally called me and told me he had the current ATIS I had asked the pilot which route he wanted to take inbound to North Las Vegas airport. Again, I failed to look at all the information in the data block and treated him as if he was VFR. When I asked the pilot which route he wanted and he was open to either route, I gave the aircraft a clearance through the Class Bravo airspace and assigned a heading of 270 with no altitude assigned. A minute or so later I asked the pilot if he was familiar with the local flying area and the surrounding terrain, and after he replied in the affirmative, I instructed the pilot to resume own navigation then fly heading 360.After I got that read back, I instructed the 

In [8]:
output = chain.invoke({"report": text})

In [9]:
output

{'categories': ['Unsafe Acts;Errors;Decision Errors',
  'Unsafe Acts;Errors;Skill-Based Errors',
  'Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State',
  'Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management',
  'Unsafe Supervisions;Inadequate Supervision']}

### 回家作業 2

你可以很清楚的看到一個飛安事故中，可以出現複數報告。
將`Report 1` 和 `Report 1.2` 結合起來產生一份的新報告。

抄也是一門技術

In [10]:
system_prompt = PromptTemplate.from_template(
    '''You are an AI assistant assigned with a task of safty report classification based on the content.
    You are a seasoned flight safety inspector with deep and extensive knowledge of aviation safty. 

    You will receieve two reports <report_1> and <report_2> and you will consolidate the content before drawing conclusion. 
    
    You always do the best work you can. You are highly analytical and pay close attention to details. 
    
    The candidates of the output are:

    - `Organizational Influence;Resource Management`
    - `Organizational Influence;Organizational Climate`
    - `Organizational Influence;rganizational Process`
    - `Unsafe Supervisions;Inadequate Supervision`
    - `Unsafe Supervisions;Planned Inappropriate Operations`
    - `Unsafe Supervisions;Failed to Correct Problem`
    - `Unsafe Supervisions;Supervisory Violation`
    - `Precondition for Unsafe Acts;Environmental Factors;Physical Environment`
    - `Precondition for Unsafe Acts;Environmental Factors;Technological Environment`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Physiological State`
    - `Precondition for Unsafe Acts;Condition of Operators;Physical/Mental Limitations`
    - `Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management`
    - `Precondition for Unsafe Acts;Personnel Factors;Personal Readiness`
    - `Unsafe Acts;Errors;Decision Errors`
    - `Unsafe Acts;Errors;Skill-Based Errors`
    - `Unsafe Acts;Errors;Perceptual Errors`
    - `Unsafe Acts;Violations;Routine`
    - `Unsafe Acts;Violations;Exceptional`

     The report can involve multiple categories.
    ''')

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

response_schemas = [
        ResponseSchema(name="categories", description="The predicted categories as a python list")
    ]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

human_prompt = PromptTemplate(template="""report_1:{report_1}; 
                                          report_2: {report_2};
                                          format instruction: {format_instructions}""",
                              input_variables=["report_1, report_2"],
                              partial_variables={'format_instructions': format_instructions}
                              )
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                               ])

chain = chat_prompt|model|output_parser

In [11]:
df.iloc[3]['Report 1.2']

'L30 TRACON Controller reported they issued VFR clearance instructions to an aircraft that was on an IFR flight plan which resulted in the aircraft flying below the Minimum Vectoring Altitude.'

In [12]:
output = chain.invoke({"report_1": text,
                       "report_2": df.iloc[3]['Report 1.2']})

In [13]:
output['categories']

['Unsafe Acts;Errors;Decision Errors',
 'Unsafe Supervisions;Inadequate Supervision',
 'Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State']

### Keynote

- 若你更理解你的數據，你可以建立更精確的Prompt，更明確的表示每個數據代表的意義，來提升輸出的品質。

# Remote server

### 1. Making a POST Request (發送 POST 請求):

- requests.post(...) sends an HTTP POST request to the specified URL.
- The URL "http://localhost:5000/openai/invoke" points to a local server running on port 5000, at the endpoint /openai/invoke.
- The json parameter is used to send a JSON payload with the request. In this case, the payload is {'input': "Where is Taiwan"}.
- requests.post(...) 發送一個 HTTP POST 請求到指定的 URL。
- URL "http://localhost:5000/openai/invoke" 指向一個本地服務器，該服務器在端口 5000 上運行，並且指向 /openai/invoke 端點。
- json 參數用於隨請求發送 JSON 負載。在這個例子中，負載是 {'input': "Where is Taiwan"}。

### 2. Response Handling (響應處理):

- The server processes the request and sends back a response.
- The response is stored in the response variable, which can then be inspected or used further in the code.
- 服務器處理請求並返回響應。
- 響應存儲在 response 變量中，之後可以檢查或在代碼中進一步使用。

In [14]:
import requests

response = requests.post(
    "http://localhost:5000/openai/invoke",
    json={'input': "Where is Taiwan"}
)

In [15]:
response.json()

{'output': {'content': 'Taiwan is an island located in East Asia. It lies off the southeastern coast of China, separated by the Taiwan Strait. To its north is Japan, and to its south is the Philippines. The island is approximately 180 kilometers (112 miles) off the coast of mainland China. Taiwan is known for its bustling cities, mountainous terrain, and rich cultural heritage. The capital city of Taiwan is Taipei.',
  'additional_kwargs': {},
  'response_metadata': {'token_usage': {'completion_tokens': 81,
    'prompt_tokens': 10,
    'total_tokens': 91,
    'prompt_tokens_details': {'cached_tokens': 0},
    'completion_tokens_details': {'reasoning_tokens': 0}},
   'model_name': 'gpt-4o-2024-05-13',
   'system_fingerprint': 'fp_d0c6e590be',
   'finish_reason': 'stop',
   'logprobs': None},
  'type': 'ai',
  'name': None,
  'id': 'run-500db718-5c13-4a60-9773-94bb8c3980f1-0',
  'example': False,
  'tool_calls': [],
  'invalid_tool_calls': [],
  'usage_metadata': None},
 'metadata': {'ru

In [16]:
response.json()['output']['content']

'Taiwan is an island located in East Asia. It lies off the southeastern coast of China, separated by the Taiwan Strait. To its north is Japan, and to its south is the Philippines. The island is approximately 180 kilometers (112 miles) off the coast of mainland China. Taiwan is known for its bustling cities, mountainous terrain, and rich cultural heritage. The capital city of Taiwan is Taipei.'

# Use the remote model as `Software as a service` (SaaS)

## Basic Usage

### 1. Creating an Instance of RemoteRunnable (創建 RemoteRunnable 的實例):

- This line creates an instance of RemoteRunnable and initializes it with the URL of the remote language model service. In this case, the service is running locally on http://localhost:5000/openai/.
- 這行代碼創建一個 RemoteRunnable 的實例，並用遠程語言模型服務的 URL 進行初始化。在這個例子中，服務在本地運行，URL 為 http://localhost:5000/openai/。

In [17]:
from langserve import RemoteRunnable

llm = RemoteRunnable("http://localhost:5000/openai/")

### 2. Asynchronous Streaming of Responses (異步流式處理回應):

- llm.astream("Where is Taiwan?") sends the query "Where is Taiwan?" to the remote service and retrieves the response as a stream.
- async for msg in ... is used to handle the streaming responses asynchronously.
- print(msg.content, end="", flush=True) prints each message content received from the stream without adding a new line after each message, and flushes the output buffer to ensure the message is displayed immediately.
- llm.astream("Where is Taiwan?") 將查詢 "Where is Taiwan?" 發送到遠程服務，並以流的形式檢索回應。
- async for msg in ... 用於異步處理流式回應。
- print(msg.content, end="", flush=True) 打印每個從流中接收到的消息內容，不在每個消息後添加新行，並刷新輸出緩衝區以確保消息立即顯示。

In [18]:
# Supports astream
async for msg in llm.astream("Where is Taiwan?"):
    print(msg.content, end="", flush=True)

Taiwan is an island located in East Asia. It lies off the southeastern coast of China, across the Taiwan Strait. To its north is Japan, and to its south is the Philippines. The island is approximately 180 kilometers (112 miles) away from the coast of mainland China. Taiwan is known for its bustling cities, mountainous terrain, and rich cultural heritage. The capital city of Taiwan is Taipei.

In [19]:
output = llm.invoke("Where is Taiwan?")

In [20]:
output.content

'Taiwan is an island located in East Asia. It lies off the southeastern coast of China, across the Taiwan Strait. To its north is Japan, and to its south is the Philippines. Taiwan is situated in the western Pacific Ocean and is part of the region known as the "Pacific Rim." The capital city of Taiwan is Taipei.'

## Make the external service a part of the chain

### 1. Comedian Chain (喜劇演員鏈)

- ChatPromptTemplate.from_messages(...) creates a prompt template where the system prompt instructs the model to either tell a joke or state a fact, and the human prompt provides the input.
- This template is then piped (|) to a language model (llm) to generate the comedian's response.
- ChatPromptTemplate.from_messages(...) 創建一個提示模板，其中系統提示指示模型要麼講一個笑話，要麼陳述一個不搞笑的事實，並且僅輸出一個。
- 然後將此模板通過管道（|）傳遞給語言模型（llm），以生成喜劇演員的回應。

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
            ('human', '{input}'
            )
        ]
    )
    | llm
)


### 2. Joke Classifier Chain

- This chain is similar to the comedian chain but serves a different purpose.
- The system prompt asks the model to classify the joke as "funny" or "not funny" and repeat the first five words for reference.
- This template is also piped to the language model (llm).
- 這個鏈與喜劇演員鏈類似，但用途不同。
- 系統提示要求模型將笑話分類為“搞笑”或“不搞笑”，並重複笑話的前五個詞以供參考。
- 此模板也通過管道傳遞給語言模型（llm）。

In [23]:
joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | llm
)


### 3. Combining Chains with RunnablePassthrough

- This combines the comedian chain and the joke classifier chain using RunnablePassthrough.assign.
- The comedian chain generates the output, and then this output is passed to the joke classifier chain to classify its humor.
- 這將喜劇演員鏈和笑話分類器鏈結合在一起，使用 RunnablePassthrough.assign。
- 喜劇演員鏈生成輸出，然後將此輸出傳遞給笑話分類器鏈以分類其幽默性。

In [24]:
chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [25]:
chain.invoke({"input": "A man and a beer"})

{'joke': AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 51, 'total_tokens': 68, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d0c6e590be', 'finish_reason': 'stop', 'logprobs': None}, id='run-4a232e70-1f6e-476f-a823-b9c1e0691927-0'),
 'classification': AIMessage(content='funny\n\n"Why did the scarecrow"', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 198, 'total_tokens': 208, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4ba85e82b6', 'finish_reason': 'stop', 'logprobs': None}, id='run-60089f8f-722c-4c80-9584-222734033019-0')}

# ChatBot

In [26]:
from IPython.display import Image

Image(url="https://python.langchain.com/v0.1/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png")

- N-Shot
- The historical chat history can be consdiered as a list of question-answer pairs
- If the chatbot doesn’t remember past chats, it’s called stateless because it doesn’t know what happened before.

## Minimal Example

In [30]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to Chinese (繁體中文): I love programming."
        )
    ]
)

AIMessage(content='我愛程式設計。', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 23, 'total_tokens': 30, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d0c6e590be', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fb66383-37f5-4086-b5d5-1c164ec69ac5-0')

In [31]:
model.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content="I haven't said anything yet! How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_1fafdb53b0', 'finish_reason': 'stop', 'logprobs': None}, id='run-128f5340-b94a-4883-b5a0-bbdcce7672d1-0')

The `memory` is not there, so it does not understand your question.

The following example shows how to add memory into the conversation.

In [33]:
model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to  Chinese (繁體中文): I love programming."
        ),
        AIMessage(content="我愛程式設計."),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I translated "I love programming" to Traditional Chinese as "我愛程式設計."', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 45, 'total_tokens': 64, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d0c6e590be', 'finish_reason': 'stop', 'logprobs': None}, id='run-211f2cb5-43db-4458-bef7-de980c3f41a8-0')

## Prompt templates

### 1. Creating a ChatPromptTemplate 

- ChatPromptTemplate.from_messages(...) creates a prompt template for the chatbot.
- The first message in the template is a system message: "You are a helpful assistant. Answer all questions to the best of your ability." This message sets the context and behavior of the assistant, instructing it to be helpful and thorough in its responses.
- MessagesPlaceholder(variable_name="messages") is a placeholder for dynamic content. The variable_name="messages" specifies that this placeholder will be filled with user messages during the conversation.
- ChatPromptTemplate.from_messages(...) 創建了一個聊天機器人的提示模板。
- 模板中的第一條消息是一條系統消息：“You are a helpful assistant. Answer all questions to the best of your ability.” 此消息設置了助手的上下文和行為，指示其在回答中要提供幫助並盡力而為。
- MessagesPlaceholder(variable_name="messages") 是一個動態內容的佔位符。variable_name="messages" 指定該佔位符將在對話中插入用戶消息。

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### 2. Creating the Chain

- This line pipes (|) the prompt template to a language model (model).
- chain represents a sequence of operations where the prompt template is used to format user messages, and the language model processes these messages to generate responses.
- 這行代碼通過管道（|）將提示模板傳遞給語言模型（model）。
- chain 代表一系列操作，其中提示模板用於格式化用戶消息，語言模型處理這些消息以生成回應。

In [35]:
chain = prompt | model

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [36]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
            content="Translate this sentence from English to  Chinese (繁體中文): I love programming."
        ),
        AIMessage(content="我愛程式設計."),
        HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I translated "I love programming" to "我愛程式設計" in Traditional Chinese (繁體中文).', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 65, 'total_tokens': 89, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_1fafdb53b0', 'finish_reason': 'stop', 'logprobs': None}, id='run-8e26ff7b-6842-4d2b-868e-e79c311ea96d-0')

## Example of Using MessageHistory

As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory

### 1. Importing the ChatMessageHistory Class (導入 ChatMessageHistory 類)

- This line imports the ChatMessageHistory class from the langchain.memory module. This class is used to handle the chat messages in memory.
- 這行代碼從 langchain.memory 模塊中導入 ChatMessageHistory 類。此類用於在內存中處理聊天消息。

In [37]:
from langchain.memory import ChatMessageHistory

### 2. Creating an Instance of ChatMessageHistory (創建 ChatMessageHistory 的實例)

- This line creates an instance of ChatMessageHistory. This instance will store the chat messages in memory for this session.
- 這行代碼創建一個 ChatMessageHistory 的實例。該實例將在此會話期間將聊天消息存儲在內存中。

In [38]:
demo_chat_history = ChatMessageHistory()

### 3. Adding User and AI Messages (添加用戶和 AI 消息)

- demo_chat_history.add_user_message("hi!") adds a user message ("hi!") to the chat history.
- demo_chat_history.add_ai_message("whats up?") adds an AI response ("whats up?") to the chat history.
- demo_chat_history.add_user_message("hi!") 將用戶消息（“hi!”）添加到聊天記錄中。
- demo_chat_history.add_ai_message("whats up?") 將 AI 回應（“whats up?”）添加到聊天記錄中。

In [39]:
demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [40]:
demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("Translate this sentence from English to  Chinese (繁體中文): I love programming.")

demo_chat_history.add_ai_message("我愛程式設計.")

demo_chat_history.messages

[HumanMessage(content='Translate this sentence from English to  Chinese (繁體中文): I love programming.'),
 AIMessage(content='我愛程式設計.')]

### 4. Retrieving the Messages (檢索消息)

- This line retrieves the list of messages stored in demo_chat_history. Each message is an object that contains information about the sender (user or AI) and the content of the message.
- 這行代碼檢索存儲在 demo_chat_history 中的消息列表。每條消息都是一個對象，包含有關發送者（用戶或 AI）和消息內容的信息。

In [41]:
{"messages": demo_chat_history.messages}

{'messages': [HumanMessage(content='Translate this sentence from English to  Chinese (繁體中文): I love programming.'),
  AIMessage(content='我愛程式設計.')]}

In [42]:
demo_chat_history.add_user_message(
    "Translate this sentence from English to  Chinese (繁體中文): I love programming."
)

response = chain.invoke({"messages": demo_chat_history.messages})

response

AIMessage(content='我愛編程。', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 76, 'total_tokens': 81, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_e40a41d203', 'finish_reason': 'stop', 'logprobs': None}, id='run-2227d1b8-f45f-4ba7-8487-9ddd55f86d46-0')

In [43]:
# Put the response back into the demo_chat_history

demo_chat_history.add_ai_message(response.content)

demo_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_chat_history.messages})

AIMessage(content='I translated "I love programming" to "我愛編程。" in Chinese (繁體中文). This is another way to say it, but if you prefer the term "程式設計," which is also commonly used, the translation would be "我愛程式設計。" Both are correct and understood in Traditional Chinese.', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 95, 'total_tokens': 163, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_e40a41d203', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1c99bb2-c0d9-4834-b9a7-cd32b62bd295-0')

# **** 預計第一個小時結束 ****

In [44]:
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from src.initialization import credential_init


credential_init()

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

C:\Users\MengChieh\Miniconda3\envs\llm_examples\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\MengChieh\Miniconda3\envs\llm_examples\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\MengChieh\Miniconda3\envs\llm_examples\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files bu

## Conversational Retrievers - Step 1

- 土味情話反殺大全 (推薦上Youtube看)

In [45]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

df = pd.DataFrame(data=[["确认过眼神，你是我爱的人。", "确认过眼神，我是你泡不到的人。"],
                         ["万水千山总是情，爱我多一点行不行。", "一寸光阴一寸金，劝你死了这条心。"],
                         ["今天吃了泡面，吃了炒面，还是想走进你的心里面。", "吃那么多面，最后还不是变成大便。"],
                         ["草莓，蓝莓，蔓越莓，今天你想我了没？", "冬瓜，西瓜，哈密瓜，你再巴巴我打得你叫妈妈。"],
                         ["众生皆苦，唯你独甜。", "尝遍众生，你为渣男代言。"],
                         ["你喜欢瑞士名表还是我帅气的外表？", "我喜欢去年买了个表。"],
                         ["我想问一条路，到哥哥心里的路。", "山路十八弯，走完脑血栓。"],
                         ["小姐姐，我心里给你留了一块地，死心塌地。", "对不起，我的心里只容得下一块地，玛莎拉蒂。"],
                         ["小姐姐你笑起来真好看啊。", "你看起来真好笑啊。"],
                         ["亲爱的你知道吗，你的笑容没有酒，我却醉得像条狗", "我的笑容没有酒，你是真的像条狗"],
                         ["宝贝儿，我在手上划了一道口子，你也划一下吧，这样我们就是两口子了", "我怕我们的血溶到一起，被你发现其实我是你爸爸"],
                         ["这世间万物都有尽头，落叶归根，而我归你", "对不起 我不收垃圾"],
                         ["请问……我想问一下路，那条通往你心里的路", "八格牙路"],
                         ["你今天怎么怪怪的？ 怪可爱的",  "你今天也怪怪的，怪恶心的"],
                         ["亲爱的，你知道我和唐僧的区别吗？ 唐僧取经我娶你", "知道你和沙僧的区别吗？ 他叫沙僧你叫沙雕"],
                         ["亲爱的，你不觉得累吗？ 你已经在我的脑海里跑了好几圈了", "傻孩子，我在找出口呢"],
                         ["莫文蔚的阴天。孙燕姿的雨天，周杰伦的晴天，都不如你和我聊天", "求求你了，能否还我一个宁静的夏天"],
                         ["如果你是方便面，那我就是白开水，今生今世，我泡定你了", "故事的最后，她变成了屎，你变成了尿，你们终究分道扬镳"],
                         ["大年三十晚上的鞭炮再响，也没有我想你那么想", "大年三十晚上的鞭炮再响，也没有你放的屁响"],
                         ["c罗可以上演帽子戏法，可我想你却没有办法", "c罗可以上演帽子戏法，我也可以给你上演绿帽子戏法"],
                         ["不要抱怨，抱我", "抱不起来，太重"],
                         ["你有没有发现我的眼睛很好看？因为我满眼都是你啊", "对不起，你眼睛在哪呢？"]], 
                  columns=['input', 'output'])

documents = []

for _, row in df.iterrows():
    documents.append(Document(page_content=row['input'], metadata={'output': row['output']}))

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 10})

## Build Chat Chain - Step 2

In [46]:
from operator import itemgetter

from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate,  MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser


system_prompt = PromptTemplate.from_template("""You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                                                You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}
                            
                                                You will reply in simplified Chinese (簡體中文). 
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                        context to the question:
                                        {context}
                                        Question: {question}
                                       """
                                      )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_template = ChatPromptTemplate.from_messages([system_message,
                                                  MessagesPlaceholder(variable_name="messages"),
                                                  human_message
                                                  ])

chain = {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "messages": itemgetter("message")} | chat_template | model | StrOutputParser()

In [ ]:
# A cleaner way of writing the pipeline.

# chain = RunnablePassthrough.assign(context=itemgetter('question')|retriever) | chat_template | model | StrOutputParser()

## Test - Step 3

https://www.wenan.wang/qibaitiaotuweiqinghua.html

In [48]:
from langchain_community.chat_message_histories import ChatMessageHistory


chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = chain.invoke({"question": question,
                           "message": chat_history.messages
                          })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)

Please input your question:  私心杂念都是你。


对不起，我的心里只容得下一块地，玛莎拉蒂。


Please input your question:  不要抱怨，抱我。


抱不起来，太重。


Please input your question:  你再不来，我要下雪了。


你再不走，我要发火了。


KeyboardInterrupt: Interrupted by user

In [49]:
chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='私心杂念都是你。'), AIMessage(content='对不起，我的心里只容得下一块地，玛莎拉蒂。'), HumanMessage(content='不要抱怨，抱我。'), AIMessage(content='抱不起来，太重。'), HumanMessage(content='你再不来，我要下雪了。'), AIMessage(content='你再不走，我要发火了。')])

### 回家作業 2: 將retriever抽換成WikipediaRetriever

基本上，你可以將這個retriever的內容抽換成任何你需要的資料，來加快寫報告的效率。記得Double Check....

## Compress the chat history to reduce the size of the prompt


https://github.com/langchain-ai/langserve/blob/main/examples/conversational_retrieval_chain/server.py

### Condensation - Step 1

In [50]:
system_prompt = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the 
                                                follow up question to be a standalone question, in its original language.
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                       Question: {question}
                                       """)

human_message = HumanMessagePromptTemplate(prompt=human_prompt)


condensed_chat_template = ChatPromptTemplate.from_messages([system_message,
                                                  MessagesPlaceholder(variable_name="messages"),
                                                  human_message
                                                  ])

condensed_chain = {"question": itemgetter("question"),
                   "messages": itemgetter("message")} | condensed_chat_template | model | StrOutputParser()

## Retrieval - Step 2

How to implement this properly?

Let start from a higher point of view

{"standalone_question": condensed_chain}|RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever}

In [52]:
from langchain_core.runnables import RunnablePassthrough


system_prompt = PromptTemplate.from_template("""You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                                                You will respond with the following style, cheesy pickup lines, shown in the context:\n\n{context}
                            
                                                You will reply in simplified Chinese (簡體中文). 
                                              """)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                        context to the question:
                                        {context}
                                        Question: {standalone_question}
                                        """
                                      )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)


retrieval = RunnablePassthrough.assign(context=itemgetter('standalone_question')|retriever)

chat_template = ChatPromptTemplate.from_messages([system_message,
                                                  human_message
                                                       ])

retrieval_chain = retrieval|chat_template|model|StrOutputParser()

In [53]:
final_chain = {"standalone_question": condensed_chain}|retrieval_chain

In [54]:
chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = final_chain.invoke({"question": question,
                                 "message": chat_history.messages
                                })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)

Please input your question:  私心杂念都是你。


私心杂念都是你？哼，听起来像是你脑子里全是些乱七八糟的想法，结果全都跟我有关。可惜啊，我对这些杂念可没兴趣。


Please input your question:  不要抱怨，抱我。


抱不起来，太重。


Please input your question:  你再不来，我要下雪了。


意思是：你再不来，我的心都要冷得像下雪一样了。哎，真是够了，怎么这么矫情呢？


Please input your question:  QUIT


## OpenAI Model Fine-tuning

剛好有人來找我家教這個，整理了一下。。。

參考:

- https://cookbook.openai.com/examples/chat_finetuning_data_prep

- {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]} {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]} {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [55]:
import json

import pandas as pd
from sklearn.model_selection import train_test_split

from src.io.path_definition import get_project_dir

jsonl = []

with open(os.path.join(get_project_dir(), 'tutorial', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

### Create a jsonl format

In [56]:
recipe_df = pd.DataFrame(data=[[rec['id'], rec['cuisine'], ", ".join(rec['ingredients'])] 
                               for rec in recipe_train], columns=['id', 'cuisine', 'ingredients'])

"""
train-test split

- 固定random state，確保數據的重現性
- 使用分層抽樣(stratified sampling)，保證訓練-測試集的class分佈是一致的
"""

train, test = train_test_split(recipe_df, test_size=0.2, random_state=42, stratify=recipe_df['cuisine'].tolist())

In [58]:
system_prompt_template = """You are a helpful AI assistant as a chef of a Michellin 3 stars restaurant. You have extensive knowledge about cuisines 
                            all over the world, and you are able to identify the origin of a cuisine based on the ingredients. You are assigned with a 
                            task of identifying the origin, as region, of cuisine based on the <ingredients>.                            
                            """

for _, row in train.iterrows():

    ingredients = row['ingredients']
    content = f"ingredients: [{ingredients}]"
    
    jsonl.append({"messages": [{"role": "system", "content": system_prompt_template}, 
                               {"role": "user", "content": content}, 
                               {"role": "assistant", "content": row['cuisine']}]})

In [60]:
# 寫入檔案

with open('openapi_finetuning_test.jsonl', 'w') as outfile:
    for entry in jsonl:
        json.dump(entry, outfile)
        outfile.write('\n')

In [61]:
# 直接使用OpenAI提供的API

from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

client.files.create?

Signature:
client.files.create(
    *,
    file: 'FileTypes',
    purpose: "Literal['assistants', 'batch', 'fine-tune', 'vision']",
    extra_headers: 'Headers | None' = None,
    extra_query: 'Query | None' = None,
    extra_body: 'Body | None' = None,
    timeout: 'float | httpx.Timeout | None | NotGiven' = NOT_GIVEN,
) -> 'FileObject'
Docstring:
Upload a file that can be used across various endpoints.

Individual files can be
up to 512 MB, and the size of all files uploaded by one organization can be up
to 100 GB.

The Assistants API supports files up to 2 million tokens and of specific file
types. See the
[Assistants Tools guide](https://platform.openai.com/docs/assistants/tools) for
details.

The Fine-tuning API only supports `.jsonl` files. The input also has certain
required formats for fine-tuning
[chat](https://platform.openai.com/docs/api-reference/fine-tuning/chat-input) or
[completions](https://platform.openai.com/docs/api-reference/fine-tuning/completions-input)
models.

T

In [ ]:
client.files.create(file=open('openapi_finetuning_test.jsonl', 'rb'),
                    purpose='fine-tune')

In [ ]:
client.fine_tuning.jobs.create?

In [ ]:
client.fine_tuning.jobs.create(model="gpt-4o-mini-2024-07-18",
                               training_file="file-SHpXusOxxMVGYRgwIGNj6mk9",
                               hyperparameters={"batch_size":4, "learning_rate_multiplier": 1e-6, "n_epochs": 5},
                               suffix='test')

訓練完之後

In [62]:
model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="ft:gpt-4o-mini-2024-07-18:cosnova-account:test:ANPq9Weh", 
                   temperature=0 # a range from 0-2, the higher the value, the higher the `creativity`
                  )

In [63]:
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate

system_prompt = PromptTemplate.from_template(system_prompt_template)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                       ingredients: [{ingredients}]
                                       """,
                              input_variables=["ingredients"]
                              )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                                ])

fine_tuned_chain = chat_prompt|model


In [64]:
fine_tuned_chain.invoke({"ingredients": test.iloc[0]['ingredients']})

AIMessage(content='The ingredients you provided—salt, whole milk, unsweetened cocoa powder, and sweet chocolate—are commonly associated with American cuisine, particularly in the context of desserts and confections. These ingredients are often used in recipes for chocolate desserts such as chocolate pudding, chocolate cake, or hot chocolate. The use of cocoa powder and sweet chocolate is especially prevalent in American baking and dessert-making traditions.', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 105, 'total_tokens': 183, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'ft:gpt-4o-mini-2024-07-18:cosnova-account:test:ANPq9Weh', 'system_fingerprint': 'fp_944baebe1a', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e4d92cd-0a60-4a57-a6fa-7d1af31ea00e-0')

In [65]:
model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini-2024-07-18", 
                   temperature=0 # a range from 0-2, the higher the value, the higher the `creativity`
                  )

chain = chat_prompt|model

In [66]:
chain.invoke({"ingredients": test.iloc[0]['ingredients']})

AIMessage(content='The ingredients you provided—salt, whole milk, unsweetened cocoa powder, and sweet chocolate—are commonly associated with American cuisine, particularly in the context of desserts like chocolate milk, chocolate pudding, or brownies. The use of cocoa powder and sweet chocolate is prevalent in various American confections and baked goods.', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 105, 'total_tokens': 167, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-b27051ee-fd59-4ffe-9e2f-8dccde2aff2d-0')